In [64]:
import pandas as pd
import numpy as np
from retinopathy.callbacks import cohen_kappa_score
from retinopathy.models.heads import regression_to_class

In [65]:
df1 = pd.read_pickle('../reg_seresnext50_rms_512_medium_mse_aptos2019_fold0_awesome_babbage_on_idrid_fold0.pkl')
df1.drop(columns=['features'], inplace=True)
df1.head()

,image_id,diagnosis_true,logits,regression
0,IDRiD_001,tensor(3),"[0.013231222, 0.008637085, 0.73025614, 0.21559...",2.241249
1,IDRiD_005,tensor(4),"[0.10114664, 0.0289896, 0.0063282703, 0.050347...",3.493390
2,IDRiD_011,tensor(3),"[0.011858935, 0.015906084, 0.9036092, 0.050163...",2.058267
3,IDRiD_014,tensor(4),"[0.007668243, 0.006203685, 0.0047766133, 0.012...",3.860499
4,IDRiD_018,tensor(2),"[0.038102895, 0.09725846, 0.7801861, 0.0401610...",1.952713


In [66]:
def regression_from_logits(logits):
    return logits[1] + logits[2] * 2 + logits[3] * 3 + logits[4] * 4

df1['regression_from_logits'] = df1['logits'].apply(regression_from_logits)
df1.head()

,image_id,diagnosis_true,logits,regression,regression_from_logits
0,IDRiD_001,tensor(3),"[0.013231222, 0.008637085, 0.73025614, 0.21559...",2.241249,2.245056
1,IDRiD_005,tensor(4),"[0.10114664, 0.0289896, 0.0063282703, 0.050347...",3.493390,3.445441
2,IDRiD_011,tensor(3),"[0.011858935, 0.015906084, 0.9036092, 0.050163...",2.058267,2.047464
3,IDRiD_014,tensor(4),"[0.007668243, 0.006203685, 0.0047766133, 0.012...",3.860499,3.928212
4,IDRiD_018,tensor(2),"[0.038102895, 0.09725846, 0.7801861, 0.0401610...",1.952713,1.955280


In [67]:
df1['diagnisos_regression'] = df1['regression'].apply(regression_to_class).apply(int)
df1['diagnisos_logits'] = df1['logits'].apply(np.argmax)
df1.head()


,image_id,diagnosis_true,logits,regression,regression_from_logits,diagnisos_regression,diagnisos_logits
0,IDRiD_001,tensor(3),"[0.013231222, 0.008637085, 0.73025614, 0.21559...",2.241249,2.245056,2,2
1,IDRiD_005,tensor(4),"[0.10114664, 0.0289896, 0.0063282703, 0.050347...",3.493390,3.445441,3,4
2,IDRiD_011,tensor(3),"[0.011858935, 0.015906084, 0.9036092, 0.050163...",2.058267,2.047464,2,2
3,IDRiD_014,tensor(4),"[0.007668243, 0.006203685, 0.0047766133, 0.012...",3.860499,3.928212,4,4
4,IDRiD_018,tensor(2),"[0.038102895, 0.09725846, 0.7801861, 0.0401610...",1.952713,1.955280,2,2


In [68]:
df1['diagnosis_true'] = df1['diagnosis_true'].apply(int)

df1['diagnisos_reg_from_log'] = df1['regression_from_logits'].apply(regression_to_class).apply(int)

In [69]:
score, num, denom = cohen_kappa_score(df1['diagnosis_true'].values, 
                          df1['diagnisos_logits'].values, 
                          weights='quadratic')
print(score)

0.759223365860179


In [70]:

score, num, denom = cohen_kappa_score(df1['diagnosis_true'].values, 
                          df1['diagnisos_regression'].values, 
                          weights='quadratic')
print(score)

0.7415271254212428


In [71]:

score, num, denom = cohen_kappa_score(df1['diagnosis_true'].values, 
                          df1['diagnisos_reg_from_log'].values, 
                          weights='quadratic')
print(score)


0.7616397302793536


In [72]:
df1[df1['diagnosis_true'] != df1['diagnisos_reg_from_log']]

,image_id,diagnosis_true,logits,regression,regression_from_logits,diagnisos_regression,diagnisos_logits,diagnisos_reg_from_log
0,IDRiD_001,3,"[0.013231222, 0.008637085, 0.73025614, 0.21559...",2.241249,2.245056,2,2,2
1,IDRiD_005,4,"[0.10114664, 0.0289896, 0.0063282703, 0.050347...",3.493390,3.445441,3,4,3
2,IDRiD_011,3,"[0.011858935, 0.015906084, 0.9036092, 0.050163...",2.058267,2.047464,2,2,2
6,IDRiD_024,4,"[0.048790153, 0.12205569, 0.3810807, 0.1345484...",2.418279,2.541963,2,2,3
7,IDRiD_030,4,"[0.028467596, 0.008369564, 0.3763195, 0.205151...",3.050488,2.903230,3,4,3
9,IDRiD_034,3,"[0.023887828, 0.014313844, 0.7780228, 0.163983...",2.056674,2.141478,2,2,2
12,IDRiD_045,2,"[0.3135545, 0.0634013, 0.509784, 0.07251287, 0...",1.617755,1.463497,2,2,1
13,IDRiD_046,4,"[0.21101043, 0.098241754, 0.64645225, 0.025946...",1.211233,1.542382,1,2,2
14,IDRiD_049,3,"[0.02046901, 0.01142912, 0.8319336, 0.10941358...",2.170901,2.110556,2,2,2
16,IDRiD_051,2,"[0.03714208, 0.8422897, 0.070583075, 0.0094649...",1.228551,1.173932,1,1,1


In [73]:

import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json

In [74]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [75]:
optR = OptimizedRounder()
optR.fit(df1['regression'], df1['diagnosis_true'])
coefficients = optR.coefficients()
print(coefficients)
df1['optimized'] = optR.predict(df1['regression'], coefficients)

score, num, denom = cohen_kappa_score(df1['diagnosis_true'].values, 
                          df1['optimized'].values, 
                          weights='quadratic')
print(score)

[0.56882927 1.59116436 2.02082512 3.03259576]
0.8005066115827089


In [76]:
optR = OptimizedRounder()
optR.fit(df1['regression_from_logits'], df1['diagnosis_true'])
coefficients = optR.coefficients()
print(coefficients)
df1['optimized'] = optR.predict(df1['regression_from_logits'], coefficients)

score, num, denom = cohen_kappa_score(df1['diagnosis_true'].values, 
                          df1['optimized'].values, 
                          weights='quadratic')
print(score)

[0.52773437 1.46015625 2.46289062 3.43164063]
0.7767686172548882


In [77]:

optR = OptimizedRounder()
optR.fit(df1['regression_from_logits'], df1['diagnosis_true'])
coefficients = optR.coefficients()
print(coefficients)
df1['optimized'] = optR.predict(df1['regression_from_logits'], coefficients)

score, num, denom = cohen_kappa_score(df1['diagnosis_true'].values, 
                          df1['optimized'].values, 
                          weights='quadratic')
print(score)


[0.52773437 1.46015625 2.46289062 3.43164063]
0.7767686172548882
